In [ ]:
#Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import csv
import re

## Data Loading and Initial Profiling

In [ ]:
# Load the dataset
# Display the first few rows to understand the data
df = pd.read_csv('Company.csv')
df.head(5)

In [ ]:
# Basic information about the dataset
df.info()

# print dataset number of rows and column
print('This dataset contains', df.shape[0], 'rows and', df.shape[1], 'columns')

In [ ]:
# Check for missing values
print("\nMissing Values per Column:")
print(df.isnull().sum())

In the above sections, the provided `Company.csv` file was loaded into a pandas DataFrame for initial exploration. Also review basic information such as missing values, duplicate entries, and a statistical summary of the dataset.

## Data Cleaning and Standardization

In [ ]:
# Trim spaces from column names
df.columns = df.columns.str.strip()

# Strip leading/trailing spaces from object-type text fields
object_cols = df.select_dtypes(include='object').columns
for col in object_cols:
    df[col] = df[col].str.strip()


In [ ]:
# Handling invalid inputs

# Get the row as a Series
row = df.iloc[11]

# Shift the values 6 positions to the right
shifted_row = row.shift(periods=6)

# Replace the original row with the shifted row
df.iloc[11] = shifted_row

# Filled company number from URI Link
df.at[11, 'CompanyNumber'] = '01477370'

# Confirm the changes
#print(df.iloc[11])


In [ ]:
# Remove columns with too many missing values (benchmark: more than 90% missing)

benchmark = 0.9
missing_ratio = df.isnull().mean()
columns_to_drop = missing_ratio[missing_ratio > benchmark].index
df_cleaned = df.drop(columns=columns_to_drop)

print(f"Columns dropped due to high missing ratio: {list(columns_to_drop)}")

In [ ]:
def clean_column_name(col):
    # Remove schema prefixes (e.g., 'RegAddress.')
    col = col.split('.')[-1]
    # Convert to snake_case
    col = re.sub(r'(?<=[a-z])(?=[A-Z0-9])|(?<=[A-Z])(?=[A-Z][a-z])', '_', col).lower()
    return col

# Apply to dataframe
df_cleaned.columns = [clean_column_name(col) for col in df_cleaned.columns]

In [ ]:
# Standardize both country and country_of_origin fields

# Define a mapping for country name standardization
country_mapping = {
    'UK': 'United Kingdom',
    'ENGLAND': 'United Kingdom',
    'england': 'United Kingdom',
    'SCOTLAND': 'United Kingdom',
    'WALES': 'United Kingdom',
    'NORTHERN IRELAND': 'United Kingdom'
    

}


# Apply mapping to country if it exists
if 'country' in df_cleaned.columns:
    df_cleaned['country'] = df_cleaned['country'].replace(country_mapping)

# Apply mapping to country_of_origin if it exists
if 'country_of_origin' in df_cleaned.columns:
    df_cleaned['country_of_origin'] = df_cleaned['country_of_origin'].replace(country_mapping)


In [ ]:

def conditional_title(x):
    if isinstance(x, str):
        return x.title()
    return x

columns_to_title = [col for col in df_cleaned.columns if col not in ['post_code', 'company_name', 'uri']]

for col in columns_to_title:
    df_cleaned[col] = df_cleaned[col].apply(conditional_title)


df_cleaned['post_code'] = df_cleaned['post_code'].str.upper()
df_cleaned['company_name'] = df_cleaned['company_name'].str.upper()
df_cleaned['company_number'] = df_cleaned['company_number'].str.upper()

## Deduplication 

In [ ]:

# 1. First, check if 'company_number' has any missing values
missing_company_numbers = df_cleaned['company_number'].isnull().sum()
print(f"Missing Company Numbers: {missing_company_numbers}")

# 2. Identify duplicate Company Numbers
duplicates = df_cleaned.duplicated(subset='company_number', keep=False)

print(f"\nNumber of Potential Duplicate Records based on company number: {duplicates.sum()}")

# 3. View duplicate records if you want to inspect
df_cleaned[duplicates].sort_values(by='company_number')

# 4. Drop duplicates, keeping the first occurrence
df_cleaned = df_cleaned.drop_duplicates(subset='company_number', keep='first')

print(f"\nDataset shape after removing duplicates: {df_cleaned.shape}")


## Setup - Companies House API Matching

In [ ]:

# Companies House API key
API_KEY = '614c43c4-f999-453e-b07d-3652f55ebf63'

def query_companies_house_by_number(company_number):
    try:
        if pd.isnull(company_number):
            return {
                'api_match_found': False,
                'api_company_number': '',
                'api_company_status': '',
                'api_incorporation_date': '',
                'api_title': '',
                'api_address_line_1': '',
                'api_address_line_2': '',
                'api_postal_code': '',
                'api_locality': '',
                'api_country': ''
            }
        
        # Call Company Profile API directly
        profile_response = requests.get(
            f'https://api.company-information.service.gov.uk/company/{company_number}',
            auth=(API_KEY, '')
        )
        
        if profile_response.status_code == 404:
            print(f"Company {company_number} not found. (404)")
            return {
                'api_match_found': False,
                'api_company_number': company_number,
                'api_company_status': '',
                'api_incorporation_date': '',
                'api_company_name': '',
                'api_address_line_1': '',
                'api_address_line_2': '',
                'api_postal_code': '',
                'api_locality': '',
                'api_country': ''
            }
        
        profile_response.raise_for_status()
        profile_data = profile_response.json()
        
        registered_address = profile_data.get('registered_office_address', {})
        
        return {
            'api_match_found': True,
            'api_company_number': company_number,
            'api_company_status': profile_data.get('company_status', ''),
            'api_incorporation_date': profile_data.get('date_of_creation', ''),
            'api_company_name': profile_data.get('company_name', ''),
            'api_address_line_1': registered_address.get('address_line_1', ''),
            'api_address_line_2': registered_address.get('address_line_2', ''),
            'api_postal_code': registered_address.get('postal_code', ''),
            'api_locality': registered_address.get('locality', ''),
            'api_country': registered_address.get('country', '')
        }
    
    except Exception as e:
        print(f"Error querying {company_number}: {e}")
        return {
            'api_match_found': False,
            'api_company_number': company_number,
            'api_company_status': '',
            'api_incorporation_date': '',
            'api_company_name': '',
            'api_address_line_1': '',
            'api_address_line_2': '',
            'api_postal_code': '',
            'api_locality': '',
            'api_country': ''
        }


In [ ]:
# Apply the updated function using company_number column
api_results = df_cleaned['company_number'].apply(query_companies_house_by_number)

# Convert to DataFrame
api_results_df = pd.json_normalize(api_results)

# Merge back with cleaned dataset
df_final = pd.concat([df_cleaned.reset_index(drop=True), api_results_df.reset_index(drop=True)], axis=1)

df_final['country'] = df_final['country'].fillna('United Kingdom')

# Display result
df_final.head()


# Matching Information Summary

In [ ]:

# Select key columns
contact_info = df_final[[
    'company_name', 'api_company_name', 'company_number','api_company_number',
    'address_line_1','api_address_line_1', 'api_address_line_2',
    'api_locality', 'post_code', 'api_postal_code', 'country', 'api_country'
]]

# Quick check on match completeness
total_records = contact_info.shape[0]
name_matches = contact_info['api_company_name'].notnull().sum()
number_matches = contact_info['api_company_number'].notnull().sum()
address_matches = contact_info['api_address_line_1'].notnull().sum()
postcode_matches = contact_info['api_postal_code'].notnull().sum()

# Print Summary
print(f"Simple Matching Summary:")
print(f"- Total Records: {total_records}")
print(f"- Company Name Matches: {name_matches} ({round(name_matches/total_records*100, 2)}%)")
print(f"- Company Name Matches: {number_matches} ({round(number_matches/total_records*100, 2)}%)")
print(f"- Address Line 1 Matches: {address_matches} ({round(address_matches/total_records*100, 2)}%)")
print(f"- Postal Code Matches: {postcode_matches} ({round(postcode_matches/total_records*100, 2)}%)")

# Display first few matched records
contact_info.head(10)


In [ ]:
# Save final matched contact information to CSV
contact_info.to_csv('final_matched_contact_info.csv', index=False)
